# 🌼 Transfer Learning com VGG16 no Oxford Flowers 102 (Google Colab)

In [ ]:

# TRANSFER LEARNING E FINE-TUNING COM VGG16 - OXFORD FLOWERS 102

## 🔁 1. Instalação de dependências
!pip install tensorflow tensorflow_datasets -q

## 📥 2. Carregando o Oxford Flowers 102
import tensorflow_datasets as tfds

dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
train_ds, val_ds, test_ds = dataset['train'], dataset['validation'], dataset['test']
num_classes = info.features['label'].num_classes

## 🧹 3. Pré-processamento (tamanho e normalização)
import tensorflow as tf

IMG_SIZE = 224
BATCH_SIZE = 32

def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, tf.one_hot(label, num_classes)

train_ds = train_ds.map(format_image).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
val_ds = val_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

## 🧠 4. Modelo com Transfer Learning (VGG16)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Congela todas as camadas

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

print("Treinando modelo com Transfer Learning (camadas congeladas)...")
history_tl = model.fit(train_ds, validation_data=val_ds, epochs=10)

## 🔓 5. Fine-Tuning (descongelando camadas finais)
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print("Treinando modelo com Fine-Tuning (últimas camadas descongeladas)...")
history_ft = model.fit(train_ds, validation_data=val_ds, epochs=5)

## 📊 6. Avaliação no conjunto de teste
loss, acc = model.evaluate(test_ds)
print(f"Acurácia final no conjunto de teste: {acc:.4f}")

## 📈 7. Gráficos comparativos
import matplotlib.pyplot as plt

def plot_history(hist1, hist2, title):
    plt.plot(hist1.history['val_accuracy'] + hist2.history['val_accuracy'], label='Val Accuracy')
    plt.plot(hist1.history['accuracy'] + hist2.history['accuracy'], label='Train Accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()

plot_history(history_tl, history_ft, "Transfer Learning + Fine-Tuning - VGG16")
